In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from typing import List


In [23]:
df = pd.read_csv('items_titles.csv')

In [13]:
df

,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...
3,Tênis Olympikus Esportivo Academia Nova Tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...
...,...
29995,Tênis Vans Old Skool I Love My Vans - Usado - ...
29996,Tênis Feminino Preto Moleca 5296155
29997,Tenis Botinha Com Pelo Via Marte Original Lanç...
29998,Tênis Slip On Feminino Masculino Original Sapa...


In [12]:
df.ITE_ITEM_TITLE.apply(lambda x: len(x.split())).describe(), df.ITE_ITEM_TITLE.apply(lambda x: len(x.split())).value_counts(normalize=True)

(count    30000.000000
 mean         7.233700
 std          2.061171
 min          1.000000
 25%          6.000000
 50%          7.000000
 75%          9.000000
 max         25.000000
 Name: ITE_ITEM_TITLE, dtype: float64,
 8     0.206867
 7     0.188333
 9     0.150533
 6     0.137567
 5     0.099200
 10    0.077800
 4     0.062867
 3     0.028733
 11    0.026233
 2     0.008333
 12    0.007800
 13    0.001700
 1     0.001333
 14    0.000700
 18    0.000433
 16    0.000333
 15    0.000267
 17    0.000267
 19    0.000200
 20    0.000167
 23    0.000133
 25    0.000100
 21    0.000067
 22    0.000033
 Name: ITE_ITEM_TITLE, dtype: float64)

Un detalle importante al momento de realizar un despliegue de modelos es poder guardar los vectorizadores y modelos ya entrenados.
Este se un ejemplo de como almacenar un vectorizador

```python
import pickle
vectorizer = CountVectorizer(ngram_range=(1,2), min_df=.05, analyzer='char')
vectorizer.fit(text)
with open('vectorizer.pk', 'wb') as f:
    pickle.dump(vectorizer, f)
```

In [118]:
class CleanText(TransformerMixin):
    '''
    Esta clase está destinada a contener todas las funciones necesarias para la limpieza del texto
    '''
    stops = stopwords.words('portuguese')    
    def fit(self, X:str, y=None):
        return self
    def _remove_stops(self, text:List)->List:
        _aux_fun = lambda x: ' '.join(list(filter(lambda x: x not in stops, x.split())))
        return list(map(_aux_fun, text))
    
    def _clean(self,  text:List, new_chars:List=[])->List:
        '''
        Función que elimina algunos símbolos, espacios repetidos y stopwords

        sent: sentencia que queremos limpar
        return: retorna la sentencia sin estos caracteres indeseados
        '''
        
        to_replace = [r'[^\w\s]', '\r', '\n', ' +'] + new_chars
        for regex in to_replace:
            for i, sentence in enumerate(text):
                text[i] =  re.sub(regex, ' ', sentence.lower()).strip()
        return text
    
    def transform(self, X:str, y=None, new_chars:List=[]) -> str:
        '''
        Dado un texto aplica la función de limpieza para eliminar símbolos básicos y comunes
        como asi también los stopwords
        '''
        sent = self._clean(X)
        sent = self._remove_stops(sent)
        return sent

class Vectorizer(TransformerMixin):
    '''
    Esta clase contiene le vectorizador que será utilizado para obtener los vectores del texto,
    reliza una vectorización a nivel de caracteres en base a la frecuencia de ocurrencia
    '''
    def fit(self, X:List[str], y=None):
        '''
        El texto con el que es entrenado será nuestro texto testigo ante el cual queremos comparar 
        nuevos títulos
        
        X: lista de sentencias para entrenar el vetorizador
        returns: vectorizador ya entrenado 
        '''
        self.count_vec = CountVectorizer(ngram_range=(1,2), min_df=.05, analyzer='char')
        return self.count_vec.fit(X)
    
    def transform(self, X:List[str], y=None):
        '''
        Toma la lista de textos y los transforma a una matriz o lista de vectores
        
        X: lista de sentencias para vectorizar
        return: np.ndarray con los vectores resultantes
        '''
        return self.count_vec.transform(X)
    
class Similarity(TransformerMixin):
    '''
    El dataset de entrenamiento será tomado como parámetro de comparación al momento
    de calcular los puntajes de similitud
    '''
    def fit(self, X:pd.Series, y=None):
        '''
        Esta función se encarga de preparar el vectorizador para nuestros textos patrones
        
        X: pd.Series con los textos que usaremos de patron
        returns: se devuelve a si mismo con los vectores ya indexados para cada texto patron
        '''
        self._vectorizer = Vectorizer().fit(X.to_list())
        self.vectors = self._vectorizer.transform(X.to_list())
        self.index = {k:vec for k, vec in zip(X.to_list(), self.vectors)}
        return self
    
    def transform(self, X:pd.Series, y=None)->pd.DataFrame:
        '''
        Función que genera el DataFrame de scores y paridad entre títulos.
        
        X: pd.Series con los textos que usaremos de comparación
        returns: pd.DataFrame con los títulos patrones y los títulos candidatos
        '''
        X_vectors = self._vectorizer.transform(X.to_list())
        index_candidate = {k:vec for k, vec in zip(X, X_vectors)}
        patrons = []
        candidates = []
        scores = []
        for title_patron in self.index:
            for title_candidate in index_candidate:
                patrons.append(title_patron)
                candidates.append(title_candidate)
                scores.append(cosine_similarity(index_candidate[title_candidate], self.index[title_patron])[0][0])
        scores = pd.DataFrame({'title_patron':patrons, 'title_candidate':candidates, 'score':scores})
        
        return scores

In [140]:
sim = Similarity().fit(df.ITE_ITEM_TITLE)

In [141]:
sm_df = sim.transform(df.ITE_ITEM_TITLE[:10])

In [142]:
sm_df[sm_df.title_candidate != sm_df.title_patron].sort_values(by='score', ascending=False)

,title_patron,title_candidate,score
156715,tênis casual masculino zarato preto 941 632,tênis casual masculino zarato 941 preto 632,1.000000
76709,sapatênis west coast couro masculino,sapatênis west coast urban couro masculino,0.974248
173483,tênis olympikus esportivo academia nova tendên...,tênis olympikus esportivo academia nova tendên...,0.966373
85788,tênis usthemp move temá tico maria vira lata 2,tênis usthemp short temá tico maria vira lata 2,0.964384
207158,tênis usthemp move temá tico maria vira lata,tênis usthemp short temá tico maria vira lata 2,0.955883
...,...,...,...
291038,dunk,tênis usthemp short temá tico maria vira lata 2,0.057496
180076,bake,tênis infantil ortopasso conforto jogging,0.057307
170469,yeezy,sapatênis west coast urban couro masculino,0.048238
101024,ês,inteligente led bicicleta tauda luz usb bicicl...,0.036226
